# Verification / Critique Pattern

In [1]:
from agentic_patterns.core.agents import get_agent, run_agent
from agentic_patterns.core.agents.utils import nodes_to_message_history

## Task: Generate a Password with Specific Constraints

We'll ask the agent to generate a password that must satisfy explicit constraints.
Then a verifier will check each constraint and report which ones pass or fail.

In [2]:
CONSTRAINTS = """
1. Exactly 12 characters long
2. Contains at least 2 uppercase letters
3. Contains at least 2 lowercase letters
4. Contains at least 2 digits
5. Contains at least 2 special characters from: !@#$%^&*
6. No consecutive repeating characters (e.g., 'aa' or '11' is not allowed)
"""

## Step 1: Generator Agent

In [3]:
generator_system_prompt = """You are a password generator. Generate passwords that satisfy the given constraints.
Output ONLY the password, nothing else."""

generator = get_agent(system_prompt=generator_system_prompt)

## Step 2: Verifier Agent

A separate agent that checks the password against each constraint explicitly.

In [4]:
verifier_system_prompt = """You are a password validator. Given a password and constraints, check each constraint.

For each constraint, output:
- PASS: [constraint] - [brief explanation]
- FAIL: [constraint] - [brief explanation]

At the end, output a line:
VERDICT: ALL_PASS or VERDICT: SOME_FAIL

Be precise and check character by character if needed."""

verifier = get_agent(system_prompt=verifier_system_prompt)

## Step 3: Initial Generation

In [5]:
generation_prompt = f"""Generate a password satisfying these constraints:
{CONSTRAINTS}
"""

gen_run, gen_nodes = await run_agent(generator, generation_prompt)
password = gen_run.result.output.strip()

print(f"Generated password: {password}")

Generated password: Tr9@bK2#mE5x


## Step 4: Verification

In [6]:
verification_prompt = f"""Verify this password against the constraints:

Password: {password}

Constraints:
{CONSTRAINTS}
"""

verify_run, _ = await run_agent(verifier, verification_prompt)
verification_result = verify_run.result.output

print("Verification result:")
print(verification_result)

Verification result:
I'll verify the password "Tr9@bK2#mE5x" against each constraint.

**Constraint 1: Exactly 12 characters long**
Counting: T-r-9-@-b-K-2-#-m-E-5-x = 12 characters
- PASS: Exactly 12 characters long - The password contains exactly 12 characters.

**Constraint 2: Contains at least 2 uppercase letters**
Uppercase letters found: T, K, E = 3 uppercase letters
- PASS: Contains at least 2 uppercase letters - Found 3 uppercase letters (T, K, E).

**Constraint 3: Contains at least 2 lowercase letters**
Lowercase letters found: r, b, m, x = 4 lowercase letters
- PASS: Contains at least 2 lowercase letters - Found 4 lowercase letters (r, b, m, x).

**Constraint 4: Contains at least 2 digits**
Digits found: 9, 2, 5 = 3 digits
- PASS: Contains at least 2 digits - Found 3 digits (9, 2, 5).

**Constraint 5: Contains at least 2 special characters from: !@#$%^&***
Special characters found: @ (position 4), # (position 8) = 2 special characters
- PASS: Contains at least 2 special chara

## Step 5: Verification Loop

If verification fails, feed the critique back to the generator and repeat.
This is the key difference from simple retry - the generator receives explicit feedback about which constraints failed.

In [7]:
MAX_ITERATIONS = 5


async def generate_with_verification(
    constraints: str, max_iterations: int = MAX_ITERATIONS
) -> tuple[str, int]:
    """Generate and verify until constraints are met or max iterations reached."""
    generator = get_agent(system_prompt=generator_system_prompt)
    verifier = get_agent(system_prompt=verifier_system_prompt)

    gen_prompt = f"Generate a password satisfying these constraints:\n{constraints}"
    gen_run, gen_nodes = await run_agent(generator, gen_prompt)
    password = gen_run.result.output.strip()

    for iteration in range(max_iterations):
        print(f"\n--- Iteration {iteration + 1} ---")
        print(f"Password: {password}")

        # Verify
        verify_prompt = f"Verify this password against the constraints:\n\nPassword: {password}\n\nConstraints:\n{constraints}"
        verify_run, _ = await run_agent(verifier, verify_prompt)
        verification = verify_run.result.output
        print(f"Verification:\n{verification}")

        # Check verdict
        if "VERDICT: ALL_PASS" in verification:
            print(f"\nSuccess after {iteration + 1} iteration(s)")
            return password, iteration + 1

        # Regenerate with feedback (continue conversation with generator)
        message_history = nodes_to_message_history(gen_nodes)
        feedback_prompt = f"""The password '{password}' failed verification:

{verification}

Generate a new password that fixes the failed constraints. Output ONLY the password."""

        gen_run, gen_nodes = await run_agent(
            generator, feedback_prompt, message_history=message_history
        )
        password = gen_run.result.output.strip()

    print("\nMax iterations reached")
    return password, max_iterations

In [8]:
final_password, iterations = await generate_with_verification(CONSTRAINTS)
print(f"\nFinal password: {final_password}")
print(f"Iterations: {iterations}")


--- Iteration 1 ---
Password: Tr8@mK5#pL9w
Verification:
I'll verify the password "Tr8@mK5#pL9w" against each constraint.

**Character-by-character breakdown:**
T-r-8-@-m-K-5-#-p-L-9-w

Position 1: T (uppercase letter)
Position 2: r (lowercase letter)
Position 3: 8 (digit)
Position 4: @ (special character)
Position 5: m (lowercase letter)
Position 6: K (uppercase letter)
Position 7: 5 (digit)
Position 8: # (special character)
Position 9: p (lowercase letter)
Position 10: L (uppercase letter)
Position 11: 9 (digit)
Position 12: w (lowercase letter)

**Constraint Verification:**

- PASS: Exactly 12 characters long - The password contains exactly 12 characters (T-r-8-@-m-K-5-#-p-L-9-w)

- PASS: Contains at least 2 uppercase letters - Found 3 uppercase letters: T (position 1), K (position 6), L (position 10)

- PASS: Contains at least 2 lowercase letters - Found 4 lowercase letters: r (position 2), m (position 5), p (position 9), w (position 12)

- PASS: Contains at least 2 digits - Found